In [ ]:
#トラカンの最近傍の値と距離を計算し、マッチさせていく(検証版)

In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import psycopg2
import datetime

In [17]:
dbname = "kddi_18_param_custom2"
npyname = "trkn_distance"
datedir = "230120"
sttime = "2022-04-01 18:00:00"

In [32]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [21]:
sql = "CREATE TABLE " + npyname + " (trknmesh1 BIGINT NULL DEFAULT NULL,trknmesh2 BIGINT NULL DEFAULT NULL,meter DOUBLE PRECISION NULL DEFAULT NULL )"
cur =connection.cursor()
cur.execute(sql)
cur.close()
connection.commit()

In [51]:
cur =connection.cursor()
meshcnt = 0
sql = "WITH gs_list AS ("
sql += "SELECT gridcode , geography( geom ) AS geom "
sql += "FROM trakanmap "
sql += ")"
sql += "SELECT "
sql += "a.gridcode,b.gridcode, MIN( ST_Distance( a.geom , b.geom ) ) "
sql += "FROM "
sql += "gs_list AS a , gs_list AS b "
sql += "WHERE "
sql += "a.gridcode <> b.gridcode AND a.gridcode <> '0' AND b.gridcode <> '0' "
sql += "GROUP BY "
sql += "a.gridcode,b.gridcode "
sql += "ORDER BY "
sql += "a.gridcode,b.gridcode ;"
cur.execute(sql)
df = pd.DataFrame(cur.fetchall())
cur.close()
cur =connection.cursor()
for index, row in df.iterrows():
    sql = "INSERT INTO " + npyname + " (trknmesh1, trknmesh2, meter) VALUES ("+ str(int(row[0])) + ","  + str(int(row[1])) + ","  + str(int(row[2])) + ");"
    cur.execute(sql)
connection.commit()
cur.close()

In [ ]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [ ]:
#特定日付のトラカンデータを全取得 > 緯度経度
cur =connection.cursor()
sql = "SELECT trakanmap.gridcode, 断面交通量,pointdata.latitude,pointdata.longitude , trakanmap.geom FROM trakan2204 "
sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
sql += "WHERE 時刻 = '2022-04-01 18:00:00' AND trakanmap.gridcode != '0' "
sql += "ORDER BY trakanmap.gridcode"
cur.execute(sql)
trknDf = pd.DataFrame(cur.fetchall())
cur.close()

In [ ]:
#特定日付のKDDIメッシュ情報を全取得
cur =connection.cursor()
sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM kddi_18_param  "
sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
sql += "WHERE times = '2022-04-01 18:00:00' AND pointdata.grid_code != '0'  ORDER BY mesh "
cur.execute(sql)
kddiDf = pd.DataFrame(cur.fetchall())
cur.close()

In [ ]:
#見つかったトラカンをループしながら、最近傍のデータとの緯度、経度を取得
#そのエリア内に含まれるKDDIメッシュ情報を取得
for trknIndex, trknRow in trknDf.iterrows():
    cur =connection.cursor()
    sql = "SELECT trknmesh2,meter,pointdata.latitude,pointdata.longitude "
    sql += "FROM trkn_distance "
    sql += "INNER JOIN pointdata ON trknmesh2 = cast(pointdata.grid_code AS BIGINT) "
    sql += "WHERE trknmesh1 = '"+trknRow[0]+"' ORDER BY meter LIMIT 1"
    cur.execute(sql)
    pathDf = pd.DataFrame(cur.fetchall())
    cur.close()
    
    cur =connection.cursor()
    sql = "SELECT trknmesh2,meter,pointdata.latitude,pointdata.longitude "
    sql += "FROM trkn_distance "
    sql += "INNER JOIN pointdata ON trknmesh2 = cast(pointdata.grid_code AS BIGINT) "
    sql += "WHERE trknmesh1 = '"+trknRow[0]+"' ORDER BY meter LIMIT 1"
    cur.execute(sql)
    pathDf = pd.DataFrame(cur.fetchall())
    cur.close()
    
#起終点のメッシュの重みはトラカン値そのままを利用
#縦と横のメッシュ数を計算
#該当のKDDIメッシュから、トラカン起終点の距離を取得
#起点までの距離と終点までの距離から比率を求める
#自身の重みを起終点から比率配分し平均化する。



In [ ]:
connection.close()